In [1]:
import sys

sys.path.append("..")

import tensorflow as tf

import numpy as np

import GPyOpt

import argparse

from utils import tf_config, preprocess_data, search_algorithms, train

from models import fmri_ae, eeg_to_fmri, uniconv_fmri

from layers import locally_connected

import matplotlib.pyplot as plt

import gc

import os


dataset="01"
memory_limit=1500
n_individuals=8
interval_eeg=6

tf_config.setup_tensorflow(device="GPU", memory_limit=memory_limit, seed=42)

with tf.device('/CPU:0'):
    train_data, _ = preprocess_data.dataset(dataset, n_individuals=n_individuals, interval_eeg=interval_eeg, verbose=True)
    _, fmri_train =train_data
    #eeg_val, fmri_val =val_data
    
    fmri_train = fmri_train[:296]

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy
I: Starting to Load Data
I: Finished Loading Data
I: Pairs Created


## Create dataset and hyperparameter definition

In [2]:
batch_size=1
learning_rate=0.001
skip_connections=True
maxpool=False
batch_norm=True
weight_decay=1e-5
n_channels=16
latent_dimension=(5,5,5)
kernel_size=(9,9,5)
stride_size=(1,1,1)
n_stacks=3

optimizer = tf.keras.optimizers.Adam(learning_rate)
loss_fn = tf.keras.losses.MSE

train_set = tf.data.Dataset.from_tensor_slices((fmri_train, fmri_train)).batch(batch_size)

## Build model

In [3]:
model = fmri_ae.fMRI_AE(latent_dimension, fmri_train.shape[1:], 
                kernel_size, stride_size, n_channels,
                maxpool=maxpool, batch_norm=batch_norm, weight_decay=weight_decay, 
                skip_connections=skip_connections, n_stacks=n_stacks, 
                local=True, local_attention=False)

## Train

In [4]:
#train
train_loss, val_loss = train.train(train_set, model, optimizer, 
                                   loss_fn, epochs=10, 
                                   val_set=None, verbose=True)

AttributeError: 'NoneType' object has no attribute 'repeat'

## Visualize brain slices

In [ ]:
from scipy.ndimage import rotate

volume=0
xslice=30
yslice=30
zslice=15

fig, axes = plt.subplots(1, 3, figsize=(15,30))
axes[0].imshow(rotate(fmri_train[volume,xslice,:,:,:], 90))
axes[1].imshow(rotate(model(fmri_train[volume:volume+1]).numpy()[volume,xslice,:,:,:], 90, axes=(0,1)))
difference = np.abs(rotate(fmri_train[volume,xslice,:,:,:], 90)-rotate(model(fmri_train[volume:volume+1]).numpy()[0,xslice,:,:,:], 90, axes=(0,1)))
axes[2].imshow(difference)
plt.show()


fig, axes = plt.subplots(1, 3, figsize=(15,30))
axes[0].imshow(rotate(fmri_train[volume,:,yslice,:,:], 90))
axes[1].imshow(rotate(model(fmri_train[volume:volume+1]).numpy()[volume,:,yslice,:,:], 90, axes=(0,1)))
difference = np.abs(rotate(fmri_train[volume,:,yslice,:,:], 90)-rotate(model(fmri_train[volume:volume+1]).numpy()[0,:,yslice,:,:], 90, axes=(0,1)))
axes[2].imshow(difference)
plt.show()


fig, axes = plt.subplots(1, 3, figsize=(15,30))
axes[0].imshow(fmri_train[volume,:,:,zslice,:])
axes[1].imshow(model(fmri_train[volume:volume+1]).numpy()[volume,:,:,zslice,:])
difference = np.abs(fmri_train[volume,:,:,zslice,:]-model(fmri_train[volume:volume+1]).numpy()[0,:,:,zslice,:])
axes[2].imshow(difference)
plt.show()

#plt.imshow(rotate(fmri_train[4,:,:,15,:], 90))

NCAONS
